In [74]:
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
OUTPUT_FILE = "results.txt"

In [123]:
import json
import os
from tqdm import tqdm


def run_on_files(func):
    file_idx = 0
    
    files = os.listdir(DATA_DIR)
    for file in tqdm(files):
        if file.startswith("judgment"):
        
            file_path = os.path.join(DATA_DIR, file)
            with open(file_path, 'r') as f:
                data = json.load(f)
                func(data)
            



In [124]:
def normalize(amount):
#     filtered_value = [character for character in amount if character not in ". "]
    
    pattern = re.compile(r'\.|\s+')
    filtered_value = re.sub(pattern, '', amount)
    joined_digits = "".join(filtered_value)
    return int(joined_digits)

In [125]:
POSSIBLE_MONEY_REGEX = r"\d[\d\.\s]*\s+zł.{3}"

ENDINGS = [
    "pt", # złptych
    "ot", # złotych
    "[^a-zA-Z]", # zł. zł, zł 
    "o-\s+tych", # zło-\ntych
]
JOINED_ENDINGS = "|".join(ending for ending in ENDINGS)

DETAILED_MONEY_REGEX = r"(?P<money>\d[\d\.\s]*)\s+zł({})".format(JOINED_ENDINGS)
print(DETAILED_MONEY_REGEX)

(?P<money>\d[\d\.\s]*)\s+zł(pt|ot|[^a-zA-Z]|o-\s+tych)


In [129]:
import re

def money_search(data):
    judgments = [x["textContent"] for x in data["items"]]
    data = "\n".join(judgments)
        
    results = re.findall(POSSIBLE_MONEY_REGEX, data, re.DOTALL)
    detailed = re.findall(DETAILED_MONEY_REGEX, data, re.DOTALL)

    
#     check for interesting edge cases which wouldn't be spotted using simple POSSIBLE_MONEY_REGEX
#     if len(results) != len(detailed):
#         for result in results:
#             if not re.findall(DETAILED_MONEY_REGEX, result):
#                 print(result)
                
    with open(OUTPUT_FILE, 'a') as f:        
        for match in detailed:
            amount, _ = match
            f.write("{}\n".format(normalize(amount)))
        

    
run_on_files(money_search)

100%|██████████| 3192/3192 [08:09<00:00,  6.52it/s]
